# Model 실행 튜토리얼

##### 1. 기본 Module

In [28]:
%pip install torch
%pip install efficientnet_pytorch
%pip install opencv-python
%pip install numpy
%pip install matplotlib
%pip install pillow
%pip install torchvision


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: C:\Users\krng0\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: C:\Users\krng0\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: C:\Users\krng0\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: C:\Users\krng0\AppData\Local\Microsoft\W

In [29]:
import torch
from efficientnet_pytorch import EfficientNet
import cv2
import numpy as np
import matplotlib as mp
from torchvision import transforms
from PIL import Image

import torch.nn as nn

##### 2. Model 불러오기
pytorch에서 모델을 저장할 때 전체 Model을 저장하는 경우와 state_dict만 저장하는 경우 존재.
1) 전체 Model을 불러오는 경우
    - torch.load(PATH)
2) state_dict로 불러오는 경우
    - model = MODEL 정의 (class, module 등) <br>
      model.load_state_dict(torch.load(PATH+MODEL_NAME))

* state_dict: 각 계층을 매개변수 텐서로 매핑되는 Python 사전(dict) 객체
* 참고자료
    - pytorch tutorial documdentation: https://tutorials.pytorch.kr/beginner/basics/saveloadrun_tutorial.html
    - justkode pytorch-save: https://justkode.kr/deep-learning/pytorch-save

In [30]:
PATH = './'
MODEL_NAME = 'best_model.pth'

class EffNet(nn.Module):
    def __init__(self, num_classes=5):
        super(EffNet, self).__init__()
        self.eff = EfficientNet.from_pretrained('efficientnet-b5', num_classes=num_classes, in_channels=1)
    def forward(self, x):
        x = self.eff(x)
        return x

# device = torch.device('cpu') # GPU 모델을 CPU에서 불러오기 위해 정의
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = EfficientNet.from_name('efficientnet-b5', num_classes=5, in_channels=1).to(device)

model = EffNet().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.load_state_dict(torch.load(PATH+MODEL_NAME, map_location=device), strict=True)


model.eval()



Loaded pretrained weights for efficientnet-b5


EffNet(
  (eff): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      1, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          48, 12, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          12, 48, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding(
     

##### 3. Testing Model

In [32]:
dir(model)
TEST_IMG = "./test_img.jpg"

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

image = Image.open(TEST_IMG)
transform(image)

# convert_tensor = transforms.ToTensor()

# processed_img = convert_tensor(image)

data = transform(image)
data = data.unsqueeze(0)

output = model(data)

RuntimeError: Numpy is not available

### 4. Result

In [71]:
dir(output)
pred = output.argmax(dim=1, keepdim=True)
output



tensor([[ 0.0018,  0.0168, -0.0030, -0.0119, -0.0068]],
       grad_fn=<AddmmBackward0>)